In [1]:
import open3d as o3d

full_pcd = o3d.io.read_point_cloud('ACMMP_model.ply')

In [2]:
o3d.visualization.draw_geometries([full_pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


2024-06-23 12:19:02.844 Python[1858:9800516] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


In [3]:
import numpy as np

def is_point_in_square(point, x_start, x_end, y_start, y_end):
    return point[0] >= x_start and point[0] < x_end and point[1] >= y_start and point[1] < y_end


def divide_pcd_into_square_segments(points, colors, num_segments):
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])

    x_step = (x_max - x_min) / num_segments
    y_step = (y_max - y_min) / num_segments

    x_grid = np.arange(x_min, x_max, x_step)
    y_grid = np.arange(y_min, y_max, y_step)
    x, y = np.meshgrid(x_grid, y_grid)

    segments = []
    for i in range(len(x_grid) - 1):
        for j in range(len(y_grid) - 1):
            # current square
            x_start, x_end = x[i, j], x[i, j + 1]
            y_start, y_end = y[i, j], y[i + 1, j]

            segment_points = []
            segment_points_colors = []
            for ind, point in enumerate(points):
                if is_point_in_square(point, x_start, x_end, y_start, y_end):
                    segment_points.append(point)
                    segment_points_colors.append(colors[ind])
            segment_points_array = np.asarray(segment_points, dtype=np.float32)
            segment_points_colors_array = np.asarray(segment_points_colors, dtype=np.float32)

            dict = {
                'points': segment_points_array,
                'colors': segment_points_colors_array
            }
            segments.append(dict)
            print("Finish: {}/{} x and {}/{} y".format(i, len(x_grid) - 1 - 1, j, len(y_grid) - 1 - 1))

    return segments

In [4]:
segment_dicts = divide_pcd_into_square_segments(np.asarray(full_pcd.points), np.asarray(full_pcd.colors), num_segments=6)

Finish: 0/4 x and 0/4 y
Finish: 0/4 x and 1/4 y
Finish: 0/4 x and 2/4 y
Finish: 0/4 x and 3/4 y
Finish: 0/4 x and 4/4 y
Finish: 1/4 x and 0/4 y
Finish: 1/4 x and 1/4 y
Finish: 1/4 x and 2/4 y
Finish: 1/4 x and 3/4 y
Finish: 1/4 x and 4/4 y
Finish: 2/4 x and 0/4 y
Finish: 2/4 x and 1/4 y
Finish: 2/4 x and 2/4 y
Finish: 2/4 x and 3/4 y
Finish: 2/4 x and 4/4 y
Finish: 3/4 x and 0/4 y
Finish: 3/4 x and 1/4 y
Finish: 3/4 x and 2/4 y
Finish: 3/4 x and 3/4 y
Finish: 3/4 x and 4/4 y
Finish: 4/4 x and 0/4 y
Finish: 4/4 x and 1/4 y
Finish: 4/4 x and 2/4 y
Finish: 4/4 x and 3/4 y
Finish: 4/4 x and 4/4 y


In [5]:
len(segment_dicts[7]['points'])

4679809

In [6]:
final_pcd_segment = o3d.geometry.PointCloud()
final_pcd_segment.points = o3d.utility.Vector3dVector(segment_dicts[7]['points'])
final_pcd_segment.colors = o3d.utility.Vector3dVector(segment_dicts[7]['colors'])
o3d.visualization.draw_geometries([final_pcd_segment])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [7]:
# перенос масок sam

In [8]:
class Camera:
    def __init__(self):
        self.K = [0.0] * 9
        self.R = [0.0] * 9
        self.t = [0.0] * 3
        self.height = 0
        self.width = 0
        self.depth_min = 0.0
        self.depth_max = 0.0

In [9]:
def read_camera(cam_path):
    camera = Camera()
    with open(cam_path, "r") as file:
        line = file.readline().strip()
        for i in range(3):
            (
                camera.R[3 * i + 0],
                camera.R[3 * i + 1],
                camera.R[3 * i + 2],
                camera.t[i],
            ) = map(float, file.readline().split())

        tmp = list(map(float, file.readline().split()))
        line = file.readline().strip()
        line = file.readline().strip()

        for i in range(3):
            camera.K[3 * i + 0], camera.K[3 * i + 1], camera.K[3 * i + 2] = map(
                float, file.readline().split()
            )

        line = file.readline().strip()
        camera.depth_min, interval, depth_num, camera.depth_max = map(
            float, file.readline().split()
        )

    return camera

In [10]:
import numpy as np
import open3d as o3d
import copy

cams_file = "images_cams/cams/00000000_cam.txt"
camera = read_camera(str(cams_file))
trans = np.asarray(
    [
        [camera.R[0], camera.R[1], camera.R[2], camera.t[0]],
        [camera.R[3], camera.R[4], camera.R[5], camera.t[1]],
        [camera.R[6], camera.R[7], camera.R[8], camera.t[2]],
        [0, 0, 0, 1],
    ]
)
pcd_segment_copy = copy.deepcopy(final_pcd_segment)
pcd_segment_copy.transform(trans)

PointCloud with 4679809 points.

In [11]:
def hidden_removal_points(pcd):
    diameter = np.linalg.norm(
        np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound())
    )
    cam = [0, 0, 0]
    radius = diameter * 100000
    _, indices = pcd.hidden_point_removal(cam, radius)
    return indices

indices = hidden_removal_points(pcd_segment_copy)
print(len(indices))

3379389


In [12]:
def get_subpcd(pcd, indices):
    subpcd = o3d.geometry.PointCloud()
    subpcd.points = o3d.utility.Vector3dVector(np.asarray(pcd.points)[indices])
    subpcd.colors = o3d.utility.Vector3dVector(np.asarray(pcd.colors)[indices])
    return subpcd

pcd_hidden_removal = get_subpcd(pcd_segment_copy, indices)
o3d.visualization.draw_geometries([pcd_hidden_removal])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [13]:
from pathlib import Path

def get_image_instances():
    masks_path = Path.cwd().joinpath(
        "geo-seg/vfm-labels/sam/00000000.npz"
    )
    return np.load(masks_path, allow_pickle=True)["masks"]

masks = get_image_instances()

In [14]:
def masks_to_image(masks):
    image_labels = np.zeros(masks[0]["segmentation"].shape)
    for i, mask in enumerate(masks):
        image_labels[mask["segmentation"]] = i + 1
    return image_labels

In [15]:
image_labels = masks_to_image(masks)

In [16]:
def get_points_to_pixels(points, img_shape, camera):
    img_width, img_height = img_shape

    intrinsic = np.asarray(
        [
            [camera.K[0], camera.K[1], camera.K[2]],
            [camera.K[3], camera.K[4], camera.K[5]],
            [camera.K[6], camera.K[7], camera.K[8]],
        ]
    )

    points_proj = intrinsic @ points.T
    points_proj[:2, :] /= points_proj[2, :]
    points_coord = points_proj.T

    inds = np.where(
        (points_coord[:, 0] < img_width)
        & (points_coord[:, 0] >= 0)
        & (points_coord[:, 1] < img_height)
        & (points_coord[:, 1] >= 0)
        & (points_coord[:, 2] > 0)
    )[0]
    print(len(inds))

    points_ind_to_pixels = {}
    for ind in inds:
        points_ind_to_pixels[ind] = points_coord[ind][:2].astype(int)

    return points_ind_to_pixels


In [17]:
points2instances = np.zeros((len(pcd_segment_copy.points), 1), dtype=int)

points_to_pixels = get_points_to_pixels(
    np.asarray(pcd_hidden_removal.points),
    ((image_labels.shape[1], image_labels.shape[0])),
    camera,
)

for point_id, pixel_id in points_to_pixels.items():
    points2instances[indices[point_id], 0] = int(
        image_labels[pixel_id[1], pixel_id[0]]
    )

2135788


In [18]:
import random

def generate_random_colors(N):
    colors = [[0, 0, 0]]
    for _ in range(N):
        colors.append(
            [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
        )

    colors = np.vstack(colors) / 255
    return colors

In [19]:
def color_pcd_by_labels(pcd, labels):
    colors = generate_random_colors(len(labels) + 1)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colored.colors = o3d.utility.Vector3dVector(
        np.zeros(np.asarray(pcd.points).shape)
    )

    for i in range(len(pcd_colored.points)):
        pcd_colored.colors[i] = colors[labels[i]]

    return pcd_colored

In [22]:
pcd_colored = color_pcd_by_labels(pcd_segment_copy, points2instances[:, 0])

In [23]:
o3d.visualization.draw_geometries([pcd_colored])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
